In [5]:
import os
os.makedirs("dataset_experiment", exist_ok=True)

In [23]:
%%writefile dataset_experiment/script.py
from azureml.core import Run, Dataset
import argparse

run = Run.get_context()
parser = argparse.ArgumentParser()
parser.add_argument("--ds", type=str, dest="dataset_name")
args = parser.parse_args()
ws = run.experiment.workspace
dataset_name = args.dataset_name
dataset = Dataset.get_by_name(name=dataset_name, workspace=ws)
df = dataset.to_pandas_dataframe()

run.log("observation_count", df.shape[0])
run.complete()

Overwriting dataset_experiment/script.py


In [24]:
from azureml.core import Experiment, Workspace, ScriptRunConfig, Environment
from azureml.core.environment import CondaDependencies

env = CondaDependencies.create(python_version="3.8",
                      pip_packages=["numpy", "pandas",
                                    "azureml-core", "azureml-defaults"])
env.save_to_file(base_directory="dataset_experiment", conda_file_path="env.yaml")
ws= Workspace.from_config()
experiment=Experiment(workspace = ws, name="dataset-experiment")
env = Environment.from_conda_specification(name="my_env", file_path="dataset_experiment/env.yaml")
src = ScriptRunConfig(source_directory="dataset_experiment", 
                      script="script.py",
                      arguments=["--ds", "wine-quality"],
                      environment = env)
run = experiment.submit(src)
run.wait_for_completion()

{'runId': 'dataset-experiment_1643030371_e5d3cae3',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2022-01-24T13:19:32.22593Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'b56f2f9a-a9a5-436a-a12d-662b8e2de423'},
 'inputDatasets': [{'dataset': {'id': '9a6cab4c-a59f-4256-a6e4-508042e11b21'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'script.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--ds', 'wine-quality'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'my_env',
   'version': 'Autosave_2022-01-24T13:15:47Z_01e862